Revising the Senate file to cover both groups of legislators.

This is going to link together three datasets: Texas Legislature Online (TLO), Texas Election Commission (TEC),
and OpenStates.

Loading TLO first.

legislators_tlo.html was copied from the source of
http://www.legis.state.tx.us/Search/BillSearchLegislatorList.aspx?ID=usrLegislatorsFolder$cboAuthor&Leg=85

In [1]:
import pandas as pd
import re
import numpy as np

pattern = """\t<option value="(A[0-9]{3,5})">([^,]+)(, (.+))? \((H|S)-A[0-9]{3,5}\)</option>"""

fp = open('../data/inputs/legislators_tlo.html', encoding = 'utf-8')
lines = fp.readlines()

leg = []

for line in lines:
    # Regex applied to each line 
    match = re.search(pattern, line)
    legislator = [match.group(1), match.group(2), match.group(4), match.group(5)]
    if legislator[3] == "H":
        legislator[3] = "STATEREP"
    else: 
        legislator[3] = "STATESEN"
    leg.append(legislator)
        

In [22]:
# Cleaning text in TLO dataset

def substitute(string):
    if type(string) == str:
        string = string.replace("&#233;", "é")
        string = string.replace("&quot;", '"')
        string = string.replace("&#225;", 'á')
        string = string.replace("&#237;", "í")
        string = string.replace("&#241;", "ñ")
        string = string.replace("\t", "")
    return string

for line in leg:
    line[1] = substitute(line[1])
    line[2] = substitute(line[2])
    if line[1] == "Lucio":
        line[1] = "Lucio Jr."


TypeError: 'str' object does not support item assignment

In [3]:
# Turning TLO dataset into a dataframe

leg = pd.DataFrame(leg, columns = ["TLO_id","last_name","first_name", "ctaSeekOfficeCd"])

In [4]:
# splitting off nicknames and suffixes in the data from TLO

def parsenickname(first):
    nicknamePattern = ' [\"\(](\w+)[\"\)]$'
    try:
        match = re.search(nicknamePattern, first)
        newLast = first.split(match.group(0))[0]
        return newLast, match.group(1)
    except:
        return first, np.nan
    
def parsesuffix(last):
    suffixPattern = " (Jr\.|Sr\.|III)$"
    try:
        match = re.search(suffixPattern, last)
        newLast = last.split(match.group(0))[0]
        return newLast, match.group(1)
    except:
        return last, np.nan

leg["first_name"], leg["filerNameShort"] = zip(*leg["first_name"].map(parsenickname))
leg["last_name"], leg["suffixes"] = zip(*leg["last_name"].map(parsesuffix))
        
leg[:5]

,TLO_id,last_name,first_name,ctaSeekOfficeCd,filerNameShort,suffixes
0,A2100,Allen,None,STATEREP,NaN,NaN
1,A2125,Alonzo,None,STATEREP,NaN,NaN
2,A2135,Alvarado,None,STATEREP,NaN,NaN
3,A2150,Anchia,None,STATEREP,NaN,NaN
4,A2155,Anderson,Charles,STATEREP,Doc,NaN


In [5]:
# This loads a file from an Open States bulk download
# Not sure that {"district": object} is the right choice, but I wanted integers with NaN allowed.

openstates = pd.read_csv("../data/inputs/openstates/2017-06-02-tx-csv/tx_legislators.csv", index_col = "leg_id",
                        dtype = {"district": object})

openstates["first_name"], openstates["filerNameShort"] = zip(*openstates["first_name"].map(parsenickname))

In [6]:
# some of the openstates rows are easy to link because they include the TLO id as part of the URL 
# to the legislator's photo.

import re

def parseTLO(url):
    urlPattern = "small\/(A[0-9]{3,5})\.jpg$"
    try:
        match = re.search(urlPattern, url)
        return match.group(1)
    except:
        return np.nan
    
openstates["TLO_id"] = (openstates["photo_url"].map(parseTLO))

In [7]:
# Finding the senators whose photo file names didn't include their TLO id.

openstates[openstates["TLO_id"].isnull()][openstates["chamber"] == "upper"]

/Users/Matt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


,full_name,first_name,middle_name,last_name,suffixes,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort,TLO_id
leg_id,,,,,,,,,,,,,,,,
TXL000198,"Eddie Lucio, Jr.",Eduardo A.,NaN,Lucio,Jr.,NaN,True,tx,upper,27,Democrat,http://www.house.state.tx.us/photos/members/36...,2010-06-19 03:51:42.083000,2017-06-01 10:38:26.283000,Eddie,NaN
TXL000312,José Menéndez,José,NaN,Menéndez,NaN,NaN,True,tx,upper,26,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.467000,2017-06-01 10:38:26.641000,NaN,NaN
TXL000705,Dawn Buckingham,Dawn,NaN,Buckingham,NaN,NaN,True,tx,upper,24,Republican,NaN,2017-01-18 03:03:18.301000,2017-06-01 10:38:26.639000,NaN,NaN
TXL000707,Borris L. Miles,Borris L.,NaN,Miles,NaN,NaN,True,tx,upper,13,Democrat,NaN,2017-01-18 03:03:18.403000,2017-06-01 10:38:26.414000,NaN,NaN
TXL000716,Bryan Hughes,Bryan,NaN,Hughes,NaN,NaN,True,tx,upper,1,Republican,NaN,2017-01-27 05:42:08.204000,2017-06-01 10:38:26.497000,NaN,NaN


In [8]:
# just setting the remaining 5 senators' TLO_id directly.

tloID = {"TXL000198": "A1300", 
        "TXL000312": "A1110",
        "TXL000705": "A1125",
        "TXL000707": "A1115",
        "TXL000716": "A1135"}

for k in tloID.keys():
    openstates.set_value(k, "TLO_id", tloID[k])


In [19]:
pd.options.display.max_rows = 200
openstates[openstates["chamber"] == "upper"]

,full_name,first_name,middle_name,last_name,suffixes,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort,TLO_id
leg_id,,,,,,,,,,,,,,,,
TXL000190,Craig Estes,Craig,NaN,Estes,NaN,NaN,True,tx,upper,30,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.063000,2017-06-01 10:38:26.268000,NaN,A1180
TXL000195,Juan Hinojosa,Juan,NaN,Hinojosa,NaN,NaN,True,tx,upper,20,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.076000,2017-06-01 10:38:26.808000,Chuy,A1250
TXL000196,Joan Huffman,Joan,NaN,Huffman,NaN,NaN,True,tx,upper,17,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.078000,2017-06-01 10:38:26.752000,NaN,A1260
TXL000198,"Eddie Lucio, Jr.",Eduardo,A.,Lucio,Jr.,NaN,True,tx,upper,27,Democrat,http://www.house.state.tx.us/photos/members/36...,2010-06-19 03:51:42.083000,2017-06-01 10:38:26.283000,Eddie,A1300
TXL000199,Jane Nelson,Jane,NaN,Nelson,NaN,NaN,True,tx,upper,12,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.086000,2017-06-01 10:38:26.740000,NaN,A1450
TXL000200,Robert Nichols,Robert,NaN,Nichols,NaN,NaN,True,tx,upper,3,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.088000,2017-06-01 10:38:26.798000,NaN,A1400
TXL000203,Kel Seliger,Kel,NaN,Seliger,NaN,NaN,True,tx,upper,31,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.095000,2017-06-01 10:38:26.762000,NaN,A1460
TXL000206,Carlos Uresti,Carlos,I.,Uresti,NaN,NaN,True,tx,upper,19,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.103000,2017-06-01 10:38:26.767000,NaN,A1605
TXL000208,Kirk Watson,Kirk,NaN,Watson,NaN,NaN,True,tx,upper,14,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.107000,2017-06-01 10:38:26.769000,NaN,A1610


In [11]:
# Trying to move Jr. to the suffix field when it occurs at the beginning of the first_name field.
# And not to overwrite suffix fields when there's no suffix in the name field.
# Probably need to use map command to take in the contents of both column and then write the contents of both.

# AFAIK, the only row affected by this is TXL000509

def parsefirstsuffix(row):
    suffixPattern = "^(Jr\.|Sr\.|III),"
    if re.match(suffixPattern, row['first_name']):
        match = re.search(suffixPattern, row['first_name'])
        newFirst = row['first_name'].split(match.group(0))[1]
        return newFirst, match.group(1)
    else:
        return row['first_name'], row['suffixes']

openstates["first_name"], openstates["suffixes"] = zip(*openstates.apply(parsefirstsuffix, axis=1))

In [16]:
# This will overwrite any existing middle name if the first name contains a space

def parsefirstmiddle(row):
    if len(row['first_name'].split()) > 1:
        return row['first_name'].split()[0], row['first_name'].split()[1]
    else:
        return row['first_name'], row['middle_name']

openstates["first_name"], openstates["middle_name"] = zip(*openstates.apply(parsefirstmiddle, axis=1))
    

In [20]:
# affects TXL000414

openstates["last_name"] = openstates["last_name"].apply(substitute)
openstates["full_name"] = openstates["full_name"].apply(substitute)

In [21]:
openstates[openstates["chamber"] == "upper"]

,full_name,first_name,middle_name,last_name,suffixes,nickname,active,state,chamber,district,party,photo_url,created_at,updated_at,filerNameShort,TLO_id
leg_id,,,,,,,,,,,,,,,,
TXL000190,Craig Estes,Craig,NaN,Estes,NaN,NaN,True,tx,upper,30,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.063000,2017-06-01 10:38:26.268000,NaN,A1180
TXL000195,Juan Hinojosa,Juan,NaN,Hinojosa,NaN,NaN,True,tx,upper,20,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.076000,2017-06-01 10:38:26.808000,Chuy,A1250
TXL000196,Joan Huffman,Joan,NaN,Huffman,NaN,NaN,True,tx,upper,17,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.078000,2017-06-01 10:38:26.752000,NaN,A1260
TXL000198,"Eddie Lucio, Jr.",Eduardo,A.,Lucio,Jr.,NaN,True,tx,upper,27,Democrat,http://www.house.state.tx.us/photos/members/36...,2010-06-19 03:51:42.083000,2017-06-01 10:38:26.283000,Eddie,A1300
TXL000199,Jane Nelson,Jane,NaN,Nelson,NaN,NaN,True,tx,upper,12,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.086000,2017-06-01 10:38:26.740000,NaN,A1450
TXL000200,Robert Nichols,Robert,NaN,Nichols,NaN,NaN,True,tx,upper,3,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.088000,2017-06-01 10:38:26.798000,NaN,A1400
TXL000203,Kel Seliger,Kel,NaN,Seliger,NaN,NaN,True,tx,upper,31,Republican,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.095000,2017-06-01 10:38:26.762000,NaN,A1460
TXL000206,Carlos Uresti,Carlos,I.,Uresti,NaN,NaN,True,tx,upper,19,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.103000,2017-06-01 10:38:26.767000,NaN,A1605
TXL000208,Kirk Watson,Kirk,NaN,Watson,NaN,NaN,True,tx,upper,14,Democrat,http://www.legdir.legis.state.tx.us/FlashCardD...,2010-06-19 03:51:42.107000,2017-06-01 10:38:26.769000,NaN,A1610


In [ ]:
tloOpen = pd.merge(leg, openstates.reset_index(), how='left', on='TLO_id')

In [ ]:
senate = tloOpen[tloOpen["ctaSeekOfficeCd"] == "STATESEN"]

In [ ]:
# This loads a file from the Texas Election Commission

tx = pd.read_csv("../data/inputs/TEC_CF_CSV/filers.csv")

In [ ]:
tx = tx[tx["filerTypeCd"] == 'COH']

In [ ]:
# removed 'STATEREP',  to focus on Senate
# removed [tx['ctaSeekOfficeCd'].isin(['STATESEN'], changed to filerHoldOfficeCd 

txRep = tx[tx['filerHoldOfficeCd'].isin(['STATESEN'])][tx['filerFilerpersStatusCd'] != "NOT_OFFICEHOLDER"]

In [ ]:
senate[:5]

In [ ]:
txRep[:5]

In [ ]:
txRep = txRep.rename(columns={"filerHoldOfficeDistrict": "district"})

# txRep["district"].describe()



In [ ]:
allThree = pd.merge(senate, txRep, how='left', on = 'district')

In [ ]:
allThree[["first_name_x", "last_name_x", "full_name", "first_name_y", "last_name_y", 
          "filerName"]]

In [ ]:
def s (string):
    return string.strip()

allThree["full_name"] = allThree["full_name"].apply(s)

In [ ]:
answer = allThree[pd.notnull(allThree['filerName'])]

In [ ]:
answer[:1]

In [ ]:
answerShort = answer[["TLO_id", "leg_id", "filerIdent", "full_name", "first_name_y", "filerNameShort", "last_name_x", "suffixes_x", "party", "district"]]

In [ ]:
answerShort = answerShort.rename(columns={"first_name_y": "first_name", "last_name_x": "last_name", 
                                "filerNameShort": "nickname", "suffixes_x": "suffixes"})

In [ ]:
answerShort["filerIdent"] = answerShort["filerIdent"].astype(int)

In [ ]:
answerShort = answerShort.set_index("TLO_id")

In [ ]:
answerShort.to_csv("../data/senateCrosswalk.csv")